# Testing mimic_III_api Routes

**App Source Code:** https://github.com/LiamWoodRoberts/mimic_III_api

**Hosted:** https://medication-ner-app.herokuapp.com/

In [1]:
# Packages
import pandas as pd
import re
import spacy
from spacy import displacy
import requests
import numpy as np
import json

**Sample Text:**

This is an 81-year-old female with a history of emphysema (not on home O2), who presents with three days of shortness of breath thought by her primary care doctor to be a COPD flare. Two days prior to admission, she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than 90%. She has also been on levofloxacin and nebulizers, and was not getting better, and presented to the [**Hospital1 18**] Emergency Room.

In [20]:
text = "This is an 81-year-old female with a history of emphysema (not on home O2), who presents with three days of shortness of breath thought by her primary care doctor to be a COPD flare. Two days prior to admission, she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than 90%. She has also been on levofloxacin and nebulizers, and was not getting better, and presented to the [**Hospital1 18**] Emergency Room."

url = "https://medication-ner-app.herokuapp.com/"
# url = "http://localhost:5000/"
r = requests.post(f"{url}api/get_tags",data = {"text":text}).json()
r

[[11, 13, 'CARDINAL', '81'],
 [48, 57, 'CONDITION', 'emphysema'],
 [71, 73, 'UNIT', 'o2'],
 [108, 127, 'SYMPTOM', 'shortness of breath'],
 [171, 175, 'CONDITION', 'copd'],
 [233, 243, 'DRUG', 'prednisone'],
 [285, 300, 'SYMPTOM', 'required oxygen'],
 [330, 347, 'UNIT', 'oxygen saturation'],
 [361, 363, 'CARDINAL', '90'],
 [387, 399, 'DRUG', 'levofloxacin'],
 [404, 414, 'DRUG', 'nebulizers']]

In [3]:
def show_ents(text,entities):
    ents = [{"start":x[0],"end":x[1],"label":x[2]} for x in entities]
    ex = [{"text":text,
       "ents":ents}
         ]
    colors = {"DRUG": "rgb(60,180,240)","DOSE":"rgb(240,180,60)","ROUTE":"rgb(200,200,200)"}
    options = {"colors":colors}
    html = displacy.render(ex,style="ent",manual=True,options=options,jupyter=True)
    return

In [4]:
show_ents(text,r)

### Available Routes:

**get_tags** - Accepts a block of text and returns entity tags and locations within the text.

**drugs_and_conditions** - Accepts a block of text and returns a list of all known drugs and conditions within the text.

**hybrid_tagger** - Accepts a block of text and returns entity tags and locations within the text.

In [5]:
def api_call(text=None,local=True,route="get_tags"):
    if text == None:
        text = input()
        
    if local:
        url = "http://localhost:5000/"
    else:
        url = "https://medication-ner-app.herokuapp.com/"
    
    r = requests.post(f"{url}api/{route}",data = {"text":text}).json()
    if route in ["get_tags","hybrid_tagger"]:
        show_ents(text,r)
    return r

In [16]:
local = False

In [17]:
api_call(text,local=local,route="get_tags")

[[11, 13, 'CARDINAL', '81'],
 [48, 57, 'CONDITION', 'emphysema'],
 [71, 73, 'UNIT', 'o2'],
 [108, 127, 'SYMPTOM', 'shortness of breath'],
 [171, 175, 'CONDITION', 'copd'],
 [233, 243, 'DRUG', 'prednisone'],
 [285, 300, 'SYMPTOM', 'required oxygen'],
 [330, 347, 'UNIT', 'oxygen saturation'],
 [361, 363, 'CARDINAL', '90'],
 [387, 399, 'DRUG', 'levofloxacin'],
 [404, 414, 'DRUG', 'nebulizers']]

In [18]:
api_call(text,local=local,route="drugs_and_conditions")

{'DRUGS': ['prednisone', 'nebulizers', 'levofloxacin'],
 'CONDITIONS': ['shortness of breath', 'required oxygen', 'emphysema', 'copd']}

In [19]:
api_call(text,local=local,route="hybrid_tagger")

[[48, 57, 'CONDITION'],
 [71, 73, 'UNIT'],
 [108, 127, 'SYMPTOM'],
 [171, 175, 'CONDITION'],
 [233, 243, 'DRUG'],
 [285, 300, 'SYMPTOM'],
 [330, 347, 'UNIT'],
 [348, 364, 'PERCENT'],
 [387, 399, 'DRUG'],
 [404, 414, 'DRUG']]

---